In [1]:
import json
from glob import glob

files = glob('texts.jsonl*.splitted.requested')
files

['texts.jsonl05.splitted.requested',
 'texts.jsonl02.splitted.requested',
 'texts.jsonl08.splitted.requested',
 'texts.jsonl06.splitted.requested',
 'texts.jsonl04.splitted.requested',
 'texts.jsonl03.splitted.requested',
 'texts.jsonl01.splitted.requested',
 'texts.jsonl07.splitted.requested',
 'texts.jsonl00.splitted.requested']

In [7]:
mapping = {}
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            mapping[l['src']] = l['r']['result']

In [8]:
len(mapping)

152630

In [9]:
data = []
with open('chatalpaca-20k.json') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [10]:
data[0]

{'id': '0',
 'conversations': [{'from': 'human',
   'value': 'Find the product of the numbers: 5 and 8'},
  {'from': 'gpt', 'value': 'The product of 5 and 8 is 40.'},
  {'from': 'human', 'value': 'What is the sum of the numbers 6 and 12?'},
  {'from': 'gpt', 'value': 'The sum of the numbers 6 and 12 is 18.'},
  {'from': 'human', 'value': 'Can you tell me the quotient of 20 and 5?'},
  {'from': 'gpt', 'value': 'Yes, the quotient of 20 and 5 is 4.'},
  {'from': 'human', 'value': 'What is the difference between 25 and 13?'},
  {'from': 'gpt', 'value': 'The difference between 25 and 13 is 12.'},
  {'from': 'human', 'value': 'What is the square of 9?'},
  {'from': 'gpt', 'value': 'The square of 9 is 81.'},
  {'from': 'human', 'value': 'What is the cube of 6?'},
  {'from': 'gpt', 'value': 'The cube of 6 is 216.'}]}

In [17]:
for i in range(len(data)):
    for k in range(len(data[i]['conversations'])):
        data[i]['conversations'][k]['value_ms'] = mapping.get(data[i]['conversations'][k]['value'])

In [18]:
data[i]['conversations']

[{'from': 'human',
  'value': 'Summarize a movie series in 2-3 sentences: Harry Potter series',
  'value_ms': 'Ringkaskan siri filem dalam 2-3 ayat: Siri Harry Potter'},
 {'from': 'gpt',
  'value': 'The Harry Potter series follows the magical adventures of young wizard Harry Potter, who embarks on a journey to save the world from the powers of dark magic. He is joined by his loyal friends Ron and Hermione as they explore the secrets of Hogwarts. Along their quest, they also must face powerful adversaries, such as the infamous Voldemort.',
  'value_ms': 'Siri Harry Potter mengikuti pengembaraan ajaib ahli sihir muda Harry Potter, yang memulakan perjalanan untuk menyelamatkan dunia daripada kuasa sihir gelap. Dia disertai oleh rakan setianya Ron dan Hermione semasa mereka meneroka rahsia Hogwarts. Sepanjang pencarian mereka, mereka juga mesti menghadapi musuh yang kuat, seperti Voldemort yang terkenal.'},
 {'from': 'human',
  'value': 'Can you remind me who played the role of Harry Potte

In [23]:
break_at = [
    'help.openai.com',
]

In [27]:
from tqdm import tqdm

all_texts = []
for i in tqdm(range(len(data))):
    texts = []
    for c in data[i]['conversations']:
        if c['from'] == 'human':
            t = '<manusia>: '
            n = c['value_ms']
            
            if n is None:
                break
            n = n.strip()
            if not len(n):
                break
            t += n
        else:
            if c['value_ms'] is None:
                break
            if any([b in c['value_ms'].lower() for b in break_at]):
                print(c['value_ms'], '\n-----\n')
                break
            n = c['value_ms']
            if n is None:
                break
            if not len(n):
                break
            n = n.strip()
            t = '<bot>: ' + n
        texts.append(t)
    if len(texts) % 2 != 0:
        texts = texts[:-1]
    if not len(texts):
        continue
    while len(texts) and texts[-1].startswith('<manusia>'):
        texts = texts[:-1]
    if not len(texts):
        continue
    if not texts[0].startswith('<manusia>'):
        continue
    all_texts.append('\n'.join(texts))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 139917.40it/s]


In [28]:
len(all_texts)

17642

In [29]:
all_texts[0]

'<manusia>: Cari hasil darab nombor: 5 dan 8\n<bot>: Hasil darab 5 dan 8 ialah 40.\n<manusia>: Apakah hasil tambah nombor 6 dan 12?\n<bot>: Jumlah nombor 6 dan 12 ialah 18.\n<manusia>: Bolehkah anda beritahu saya hasil bagi 20 dan 5?\n<bot>: Ya, hasil bagi 20 dan 5 ialah 4.\n<manusia>: Apakah perbezaan antara 25 dan 13?\n<bot>: Perbezaan antara 25 dan 13 ialah 12.\n<manusia>: Apakah kuasa dua bagi 9?\n<bot>: Kuasa dua bagi 9 ialah 81.\n<manusia>: Apakah kubus 6?\n<bot>: Kubus 6 ialah 216.'

In [30]:
with open('chatalpaca_texts.jsonl', 'w') as fopen:
    for t in all_texts:
        fopen.write(f'{json.dumps(t)}\n')